In [1]:
!pip install jiwer -q
!pip install torchaudio -q
!pip install git+https://github.com/huggingface/transformers.git -q
!pip install librosa==0.8.1 -q
!pip install git+https://github.com/huggingface/datasets.git -q
!pip install transformers==4.21.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 29.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 77.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
   

In [2]:
!mkdir -p /content/data
file_id = "1F5fEFH6bO-eS3jhxeLbwUVlENBL1JrH1"
!gdown --id $file_id -O /content/data/audio.zip
!unzip -q /content/data/audio.zip -d /content/data/

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1F5fEFH6bO-eS3jhxeLbwUVlENBL1JrH1
To: /content/data/audio.zip
100% 208M/208M [00:04<00:00, 48.1MB/s]


In [3]:
!%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=/content/cache
%env HF_DATASETS_CACHE=/content/cache
%env CUDA_LAUNCH_BLOCKING=1

/bin/bash: line 1: fg: no job control
env: LANG=C.UTF-8
env: TRANSFORMERS_CACHE=/content/cache
env: HF_DATASETS_CACHE=/content/cache
env: CUDA_LAUNCH_BLOCKING=1


In [4]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split

import os
import sys

import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

#Подготовка данных

Filename identifiers

* Modality (01 = full-AV, 02 = video-only, 03 = audio-only).

* Vocal channel (01 = speech, 02 = song).
* Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
* Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
* Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
* Repetition (01 = 1st repetition, 02 = 2nd repetition).
* Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).


In [5]:
import os
import pandas as pd

# Путь к каталогу с данными
data_dir = '/content/data'

# Создание словаря для соответствия номеров эмоций и их названий
emotion_mapping = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

# Создание пустого списка для хранения данных
data = []

# Перебор всех папок актеров
for actor_folder in os.listdir(data_dir):
    actor_path = os.path.join(data_dir, actor_folder)

    # Проверка, что actor_path - это действительно каталог
    if os.path.isdir(actor_path):
        # Перебор всех файлов .wav внутри папки актера
        for wav_file in os.listdir(actor_path):
            if wav_file.endswith(".wav"):
                file_name = os.path.splitext(wav_file)[0]  # Извлечение имени файла без расширения
                file_path = os.path.join(actor_path, wav_file)  # Полный путь до файла
                emotion_code = file_name.split("-")[2]  # Извлечение кода эмоции из имени файла
                emotion = emotion_mapping.get(emotion_code, 'unknown')  # Получение названия эмоции из словаря
                data.append([file_name, file_path, emotion])

# Создание DataFrame
df = pd.DataFrame(data, columns=['Название файла', 'Путь', 'Эмоция'])

In [6]:
df.head()

,Название файла,Путь,Эмоция
0,03-01-03-01-01-02-20,/content/data/Actor_20/03-01-03-01-01-02-20.wav,happy
1,03-01-07-02-01-01-20,/content/data/Actor_20/03-01-07-02-01-01-20.wav,disgust
2,03-01-08-02-02-02-20,/content/data/Actor_20/03-01-08-02-02-02-20.wav,surprised
3,03-01-06-01-02-01-20,/content/data/Actor_20/03-01-06-01-02-01-20.wav,fearful
4,03-01-07-02-02-02-20,/content/data/Actor_20/03-01-07-02-02-02-20.wav,disgust


In [7]:
len(df)

1440

In [8]:
# Подсчет количества строк для каждой эмоции
emotion_counts = df['Эмоция'].value_counts().reset_index()
emotion_counts.columns = ['Эмоция', 'Количество строк']

# Вывод результата в виде DataFrame
emotion_counts

,Эмоция,Количество строк
0,happy,192
1,disgust,192
2,surprised,192
3,fearful,192
4,angry,192
5,calm,192
6,sad,192
7,neutral,96


Нейтральных эмоций меньше по условию "There is no strong intensity for the 'neutral' emotion."

Послушаем некоторые сэмплы

In [9]:
idx = np.random.randint(0, len(df))
sample = df.iloc[idx]
path = sample["Путь"]
label = sample["Эмоция"]


print(f"ID Location: {idx}")
print(f"      Label: {label}")
print()

speech, sr = torchaudio.load(path)
speech = speech[0].numpy().squeeze()
# Ресемплинг в 16 кГц
speech = librosa.resample(np.asarray(speech), sr, 16_000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

ID Location: 397
      Label: angry



## Разделение выборки на тренировочную и тестовую

In [10]:
save_path = "/content/data"

train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df["Эмоция"])

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)


print(train_df.shape)
print(test_df.shape)

(1152, 3)
(288, 3)


In [11]:
# Подсчет количества строк для каждой эмоции
emotion_counts = train_df['Эмоция'].value_counts().reset_index()
emotion_counts.columns = ['Эмоция', 'Количество строк']

# Вывод результата в виде DataFrame
emotion_counts

,Эмоция,Количество строк
0,disgust,154
1,angry,154
2,sad,154
3,happy,154
4,calm,153
5,fearful,153
6,surprised,153
7,neutral,77


In [12]:
from datasets import load_dataset, load_metric


data_files = {
    "train": "/content/data/train.csv",
    "validation": "/content/data/test.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

print(train_dataset)
print(eval_dataset)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['Название файла', 'Путь', 'Эмоция'],
    num_rows: 1152
})
Dataset({
    features: ['Название файла', 'Путь', 'Эмоция'],
    num_rows: 288
})


In [13]:
input_column = "Название файла"
output_column = "Эмоция"

In [14]:
label_list = train_dataset.unique(output_column)
label_list.sort()
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

A classification problem with 8 classes: ['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']


In [15]:
from transformers import AutoConfig, Wav2Vec2Processor

In [16]:
model_name_or_path = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
# В дальнейшем для обучения на русском: jonatasgrosman/wav2vec2-large-xlsr-53-russian
pooling_mode = "mean"

In [17]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

In [18]:
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

The target sampling rate: 16000


In [20]:
import os
import librosa
import numpy as np

target_sampling_rate = 16000

def get_file_path(actor_number, file_name):
    actor_folder = f"Actor_{actor_number:02d}"
    return os.path.join("/content/data", actor_folder, f"{file_name}.wav")

def speech_file_to_array_fn(actor_number, file_name):
    file_path = get_file_path(actor_number, file_name)
    speech_array, sampling_rate = librosa.load(file_path, sr=target_sampling_rate)

    return speech_array

def preprocess_function(examples):
    actor_numbers = [int(path.split("-")[6]) for path in examples[input_column]]
    speech_list = [speech_file_to_array_fn(actor_number, path) for actor_number, path in zip(actor_numbers, examples[input_column])]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]

    result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result

def label_to_id(label, label_list):
    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1
    return label

In [21]:
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=4
)

Map (num_proc=4):   0%|          | 0/1152 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different length

In [22]:
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=4
)

Map (num_proc=4):   0%|          | 0/288 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different length

In [ ]:
# idx = 0
# print(f"Training input_values: {train_dataset[idx]['input_values']}")
# print(f"Training attention_mask: {train_dataset[idx]['attention_mask']}")
# print(f"Training labels: {train_dataset[idx]['labels']} - {train_dataset[idx]['Эмоция']}")

#Модель

In [24]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [25]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)


class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


#Тренировка

In [26]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor


@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

In [27]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [28]:
is_regression = False

In [29]:
import numpy as np
from transformers import EvalPrediction


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [30]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english were not used when initializing Wav2Vec2ForSpeechClassification: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this mode

In [31]:
model.freeze_feature_extractor()

In [32]:
import torch
print(torch.cuda.is_available())

True


In [33]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/wav2vec2-xlsr-english-speech-emotion-recognition",
    # output_dir="/content/gdrive/MyDrive/wav2vec2-xlsr-english-speech-emotion-recognition"
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2, # больше - лучше
    evaluation_strategy="steps",
    num_train_epochs=2.0,
    fp16=True,
    save_steps=10,
    eval_steps=10,
    logging_steps=20,
    learning_rate=1e-4,
    save_total_limit=2,
)

In [34]:
tokenizer=processor.feature_extractor
tokenizer.save_pretrained(training_args.output_dir)

['/content/wav2vec2-xlsr-english-speech-emotion-recognition/preprocessor_config.json']

In [35]:
# !pip install packaging -q
# !git clone https://github.com/NVIDIA/apex
# %cd apex
# !pip install -q -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [36]:
from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)

if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast


class CTCTrainer(Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:

        model.train()
        inputs = self._prepare_inputs(inputs)

        if self.use_cuda_amp :
            with autocast():
                loss = self.compute_loss(model, inputs)
        else:
            loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if self.use_cuda_amp :
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()


In [37]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


#Тренировка

In [38]:
print(dir(CTCTrainer))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_gather_and_numpify', '_get_collator_with_removed_columns', '_get_eval_sampler', '_get_learning_rate', '_get_train_sampler', '_hp_search_setup', '_inner_training_loop', '_issue_warnings_after_load', '_load_best_model', '_load_from_checkpoint', '_load_optimizer_and_scheduler', '_load_rng_state', '_maybe_log_save_evaluate', '_move_model_to_device', '_nested_gather', '_pad_across_processes', '_prepare_input', '_prepare_inputs', '_push_from_checkpoint', '_remove_unused_columns', '_report_to_hp_search', '_rotate_checkpoints', '_save', '_save_checkpoint', '_save_tpu', '_set_signature_columns_if_needed', '_sorted_checkpoints', '_tune_save_checkpoint', '

In [39]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: Эмоция, Путь, Название файла. If Эмоция, Путь, Название файла are not expected by `Wav2Vec2ForSpeechClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1152
  Num Epochs = 2
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 576


Step,Training Loss,Validation Loss,Accuracy
10,No log,2.086985,0.187500
20,2.125500,2.087440,0.131944
30,2.125500,2.080841,0.173611
40,2.039800,2.037089,0.236111
50,2.039800,1.950041,0.236111
60,2.035700,1.828581,0.288194
70,2.035700,1.775960,0.326389
80,1.771300,1.722886,0.350694
90,1.771300,1.776510,0.350694
100,1.706100,2.062860,0.274306


The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: Эмоция, Путь, Название файла. If Эмоция, Путь, Название файла are not expected by `Wav2Vec2ForSpeechClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 288
  Batch size = 2
Saving model checkpoint to /content/wav2vec2-xlsr-english-speech-emotion-recognition/checkpoint-10
Configuration saved in /content/wav2vec2-xlsr-english-speech-emotion-recognition/checkpoint-10/config.json
Model weights saved in /content/wav2vec2-xlsr-english-speech-emotion-recognition/checkpoint-10/pytorch_model.bin
Feature extractor saved in /content/wav2vec2-xlsr-english-speech-emotion-recognition/checkpoint-10/preprocessor_config.json
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: Эмоция, Путь, Название файла. 

TrainOutput(global_step=576, training_loss=1.1460684339205425, metrics={'train_runtime': 3990.6357, 'train_samples_per_second': 0.577, 'train_steps_per_second': 0.144, 'total_flos': 2.7230453248576464e+17, 'train_loss': 1.1460684339205425, 'epoch': 2.0})

#Оценка

In [40]:
import librosa
from sklearn.metrics import classification_report

In [41]:
test_dataset = load_dataset("csv", data_files={"test": "/content/data/test.csv"}, delimiter="\t")["test"]
test_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['Название файла', 'Путь', 'Эмоция'],
    num_rows: 288
})

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [43]:
# Сохранение модели
processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")
processor.save_pretrained('model_directory')
model.save_pretrained('model_directory')
tokenizer.save_pretrained('model_directory')

loading feature extractor configuration file https://huggingface.co/jonatasgrosman/wav2vec2-large-xlsr-53-english/resolve/main/preprocessor_config.json from cache at /content/cache/fc6796f7821647c9688e7962f0ec34203c203d8bb6776a47ad1bcf0306eaa10c.ea4f8997cad703518b6b984d48b5d4a3cac9824e668cdf5b7bcda5734560c5e5
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2ProcessorWithLM",
  "return_attention_mask": true,
  "sampling_rate": 16000
}

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/jonatasgrosman/wav2vec2-large-xlsr-53-english/resolve/main/config.json from cache at /content/cache/d38103e58c39973cfd1e2ec80562af01ab4c06f185ec158a79b5fa19a4b63202.ae2ca42ecd89a197c6c99fe5e901d7cc4f76736d332608c12ff5d1f9e4ef00ae
Model conf

['model_directory/preprocessor_config.json']

In [44]:
model_name_or_path = "/content/model_directory"
config = AutoConfig.from_pretrained(model_name_or_path)
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

loading configuration file /content/model_directory/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "/content/model_directory",
  "activation_dropout": 0.05,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForSpeechClassification"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": true,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropou

In [45]:
# from google.colab import drive
# drive.mount('/content/drive')

In [46]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["Путь"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)

    batch["speech"] = speech_array
    return batch


def predict(batch):
    features = processor(batch["speech"], sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits

    pred_ids = torch.argmax(logits, dim=-1).detach().cpu().numpy()
    batch["predicted"] = pred_ids
    return batch

In [48]:
test_dataset = test_dataset.map(speech_file_to_array_fn)

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

ArrowInvalid: ignored

In [49]:
result = test_dataset.map(predict, batched=True, batch_size=8)

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

KeyError: ignored

In [ ]:
label_names = [config.id2label[i] for i in range(config.num_labels)]
label_names

In [ ]:
y_true = [config.label2id[name] for name in result["Эмоция"]]
y_pred = result["predicted"]

print(y_true[:5])
print(y_pred[:5])

In [ ]:
print(classification_report(y_true, y_pred, target_names=label_names))

In [ ]:
import shutil

shutil.make_archive("/content/my_cool_model", 'zip', "/content/model_directory")

# Скачать архив
from google.colab import files
files.download('/content/my_cool_model.zip')

#Предсказание вручную

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from transformers import AutoConfig, Wav2Vec2Processor

import librosa
import IPython.display as ipd
import numpy as np
import pandas as pd

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name_or_path = "/content/model_directory"
config = AutoConfig.from_pretrained(model_name_or_path)
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
sampling_rate = processor.feature_extractor.sampling_rate
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

In [ ]:
def speech_file_to_array_fn(path, sampling_rate):
    speech_array, _sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech


def predict(path, sampling_rate):
    speech = speech_file_to_array_fn(path, sampling_rate)
    features = processor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits

    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = [{"Emotion": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
    return outputs


def prediction(df_row):
    path, emotion = df_row["Путь"], df_row["Эмоция"]
    df = pd.DataFrame([{"Emotion": emotion, "Sentence": "    "}])
    setup = {
        'border': 2,
        'show_dimensions': True,
        'justify': 'center',
        'classes': 'xxx',
        'escape': False,
    }
    ipd.display(ipd.HTML(df.to_html(**setup) + "<br />"))
    speech, sr = torchaudio.load(path)
    speech = speech[0].numpy().squeeze()
    speech = librosa.resample(np.asarray(speech), sr, sampling_rate)
    ipd.display(ipd.Audio(data=np.asarray(speech), autoplay=True, rate=sampling_rate))

    outputs = predict(path, sampling_rate)
    r = pd.DataFrame(outputs)
    ipd.display(ipd.HTML(r.to_html(**setup) + "<br />"))

In [ ]:
predict('/content/data/Actor_12/03-01-05-02-02-02-12.wav', 16000)

In [ ]:
test = pd.read_csv("/content/data/test.csv", sep="\t")
test.head(5)

In [ ]:
prediction(test.iloc[0])